<h1><center> PPOL 5203 : Final Project  <br><br> 
<font color='grey'> Cleaning Capital Bikeshare Data <br><br>
Ibadat Jarg and Helen Wang</center></center> <h1> 

In [ ]:
#Download Librries as neened
#pip install numpy
#pip install pandas
#!pip install GeoJSON

In [ ]:
#Setting up libraries
import numpy as np
import pandas as pd
import json
import os

In [ ]:
#Importing json file
with open(r'..\data\raw_data\capital_bike_Locations.geojson', 'r') as f:
    data = json.load(f)
    

In [ ]:
#grabbing features from data
features = data['features']

In [ ]:
# Convert to Pandas DataFrame
# Include 'properties' for attributes and optionally 'geometry' for coordinates
main_df = pd.DataFrame([{
    **feature['properties'],  # Unpack properties
    'geometry': feature['geometry']  # Include geometry (optional)
} for feature in features])


In [ ]:
print(main_df.head())

In [ ]:
### Script to read all csv files and combine them into one data frame

# Directory containing CSV files - links provided in github since size of files is too large. Path provided here is local
directory_path = r"C:\Users\Ibadat\Desktop\PPOL5203 Data Science 1\final_project\bikeshare_ride_history"

In [ ]:
# List to hold dataframes
dataframes = []

In [ ]:
# Traverse the directory tree
for root, dirs, files in os.walk(directory_path):
    for filename in files:
        if filename.lower().endswith(".csv"):  # Handle case-insensitivity for CSV files
            file_path = os.path.join(root, filename)
            print(f"Reading {file_path}...")
            try:
                # Read each CSV file and append to the list
                dataframes.append(pd.read_csv(file_path))
            except Exception as e:
                print(f"Failed to read {file_path}: {e}")

In [ ]:
# Combine all dataframes into a single dataframe
if dataframes:  # Ensure there are dataframes to concatenate
    combined_df = pd.concat(dataframes, ignore_index=True)

    # Save combined dataframe to a new CSV (optional)
    output_file = os.path.join(directory_path, "combined_output.csv")
    combined_df.to_csv(output_file, index=False)

    print(f"Combined CSV saved to {output_file}.")
else:
    print("No CSV files found in the directory or its subfolders.")

In [ ]:
#Setting file path to read the combined data (could skip this if elected not to save output in separate file)
file_path = r"C:\Users\Ibadat\Desktop\PPOL5203 Data Science 1\final_project\bikeshare_ride_history\combined_output.csv"

#Saving as df
df = pd.read_csv(file_path)

In [ ]:
#Filling in missing values for start date
df['Start'] = df['Start date'].fillna(df['started_at'])

In [ ]:
#Filling in missing values for start date
df['Name'] = df['Start station'].fillna(df['start_station_name'])

In [ ]:
#Now we have station name and start date for all stations and can identify which year the capital bikeshare location was openned

#Confirming that the format of 'Start' is correct
df['Start'] = pd.to_datetime(df['Start'])

In [ ]:
#Grouping by the lowest values in the data column
earliest_dates = df.groupby('Name')['Start'].min()

In [ ]:
# Create a new DataFrame for opening years
opening_years = earliest_dates.dt.year.reset_index()
opening_years.rename(columns={'Start': 'Opening Year'}, inplace=True)

In [ ]:
# Merge the opening year back into the original DataFrame
df = df.merge(opening_years, on='Name', how='left')

In [ ]:
# Display the updated DataFrame
print(df.head())

In [ ]:
# Ensure 'df' has unique 'Name' and 'Opening Year'
df_unique = df.drop_duplicates(subset=['Name'])[['Name', 'Opening Year']]

In [ ]:
# Merge main_df with the unique 'Opening Year' from df, specifying the column names
main_df = main_df.merge(df_unique, left_on='NAME', right_on='Name', how='left')

In [ ]:
# Display the updated main_df
print(main_df.head())

In [ ]:
#saving file
main_df.to_csv("../data/cleaned_data/opened_capital_bikes.csv", index = False)